# Extracting human understandable insights from any Machine Learning model

![](https://miro.medium.com/max/639/1*tX9jrdkCZm3jPqG0rWXwOQ.png)
[Source: interpretable-ml-book](https://christophm.github.io/interpretable-ml-book/terminology.html)

In his book [‘Interpretable Machine Learning’](https://christophm.github.io/interpretable-ml-book/), Christoph Molnar beautifully encapsulates the essence of ML interpretability through this example: Imagine you are a Data Scientist and in your free time you try to predict where your friends will go on vacation in the summer based on their facebook and twitter data you have. Now, if the predictions turn out to be accurate, your friends might be impressed and could consider you to be a magician who could see the future.


If the predictions are wrong, it would still bring no harm to anyone except to your reputation of being a “Data Scientist”. Now let’s say it wasn’t a fun project and there were investments involved. Say, you wanted to invest in properties where your friends were likely to holiday. What would happen if the model’s predictions went awry?You would lose money. As long as the model is having no significant impact, it’s interpretability doesn’t matter so much but when there are implications involved based on a model’s prediction, be it financial or social, interpretability becomes relevant.


## Importance of interpretability

The question that some of the people often ask is why aren’t we just content with the results of the model and why are we so hell-bent on knowing why a particular decision was made? A lot of this has to do with the impact that a model might have in the real world. For models which are merely meant to recommend movies will have a far less impact than the ones created to predict the outcome of a drug.

![](https://miro.medium.com/max/1000/0*IWLWbvl2xUJLD-va.png)

[The big picture of explainable machine learning.](https://christophm.github.io/interpretable-ml-book/agnostic.html)

Some of the  [benefits that interpretability](https://www.kaggle.com/dansbecker/use-cases-for-model-insights)  brings along are:

![](https://imgur.com/nng8iT1.png)


# Model Explainability techniques

Let’s discuss a  few  techniques which help in extracting the above insights from a model. These techniques in the order in which they are discussed are as follows:

<div class="alert alert-block alert-info">

<h4>ELI5 library</h3>
<h4>Partial Dependence Plots</h3>
<h4>SHAP Values</h3>
<h4>Advanced Uses of SHAP Values</h3>
<h4>SHAP Summary Plots</h3>
<h4>LIME</h3>
</div>




## Objective

The objective of the notebook is to use techniques to explain how different features have different effect on the prediction whether or not a patient has diabetes, based on certain diagnostic measurements.


In [ ]:
# Loading necessary libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")      

In [ ]:
# Reading in the data
df = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
df.head()


In [ ]:
df.info()

In [ ]:
# Creating the target and the features column and splitting the dataset into test and train set.

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# 1. Permutation Importance using ELI5 library

What features does a model think are important?  Which features might have a greater impact on the model predictions than the others?  This concept is called feature importance and **Permutation** **Importance** is a technique used widely for calculating feature importance.  It helps us to see when our model produces  counterintuitive  results, and it helps to show the others when our model is working as we’d hope.

Permutation Importance works for many scikit-learn estimators. The idea is simple: Randomly permutate or shuffle a single column in the validation dataset leaving all the other columns intact.  A feature is considered “important” if the model’s accuracy drops a lot and causes an increase in error. On the other hand, a feature is considered ‘unimportant’ if shuffling its values don’t affect the model’s accuracy.

* Permutation importance is useful useful for debugging, understanding your model, and communicating a high-level overview from your model.

* Permutation  importance is calculated  after a model has been fitted. So, let’s train and fit a  **RandomForestClassifier**  model denoted as  **my_model**  on the training data.

* * Permutation Importance is calculated using the **ELI5 library**.  [**ELI5**](https://github.com/TeamHG-Memex/eli5) is a Python library which allows to visualize and debug various Machine Learning models using unified API. It has built-in support for several ML frameworks and provides a way to explain black-box models.


In [ ]:
# Training and fitting a Random Forest Model
my_model = RandomForestClassifier(n_estimators=100,
                                  random_state=0).fit(X_train, y_train)

In [ ]:
# Calculating and Displaying importance using the eli5 library
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(my_model, random_state=1).fit(X_test,y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

**Interpretation**

-   The features at the top are most important and at the bottom, the least. For this example, 'Glucose level' is the most important feature which decides whether a person will have diabetes, which also makes sense.
-   The number after the  **±**  measures how performance varied from one-reshuffling to the next.
-   Some weights are negative. This is because in those cases predictions on the shuffled data  were  found to be more accurate than the real data.


# 2. Partial Dependence Plots

The partial dependence plot (short PDP or PD plot) shows the marginal effect one or two features have on the predicted outcome of a machine learning model( [J. H. Friedman 2001](https://statweb.stanford.edu/~jhf/ftp/trebst.pdf)). PDPs show how a feature affects predictions. PDP can show the relationship between the target and the selected features via 1D or 2D plots.



In [ ]:
# training and fitting a Decision Tree
from sklearn.tree import DecisionTreeClassifier
feature_names = [i for i in df.columns]
tree_model = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

In [ ]:
feature_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']

In [ ]:
# Let's plot a decision tree source : #https://www.kaggle.com/willkoehrsen/visualize-a-decision-tree-w-python-scikit-learn
# Since there are a lot of attributes, it is difficult to actually make sense of the decision tree graph in this notebook. 
# It is advised to export it as png and view it.

from sklearn import tree
import graphviz

tree_graph = tree.export_graphviz(tree_model, out_file=None, feature_names=feature_names,filled = True)
graphviz.Source(tree_graph)

The library to be used for plotting PDPs is called  **python partial dependence plot toolbox** or simply [**PDPbox**]. 
PDPs are also calculated after a model has been fit. In our dataset there are a lot of features like Glucose, BLood Pressure, Age etc. We start by considering a single row. 

We proceed by fitting our model and calculating the probability of a person having diabetes which is our target variable. Next, we would choose a variable and continuously alter its value. For instance, we will calculate the outcome if the person's insulin level is 50,100,150 and so on. All these values are then plotted and we get a graph of predicted  vs actual outcome.(https://pdpbox.readthedocs.io/en/latest/)

In [ ]:

from pdpbox import pdp, get_dataset, info_plots

# Create the data that we will plot
pdp_goals = pdp.pdp_isolate(model=tree_model, dataset=X_test, model_features=feature_names, feature='Glucose')

# plot it
pdp.pdp_plot(pdp_goals, 'Glucose')
plt.show()

In [ ]:
# Create the data that we will plot
pdp_goals = pdp.pdp_isolate(model=tree_model, dataset=X_test, model_features=feature_names, feature='Insulin')

# plot it
pdp.pdp_plot(pdp_goals, 'Insulin')
plt.show()

**Interpretation**

-   The Y-axis represents the change in prediction from what it would be predicted at the baseline or leftmost value.
-   Blue area denotes the confidence interval
-   For the ‘Glucose’ graph, we  observe  that probability of a person having diabetes steeply increases as the glucose level goes beyond 140 and then the probability remains high. 



**We can also visualize the partial dependence of two features at once using 2D Partial plots.**

In [ ]:
features_to_plot = ['Glucose','Insulin']
inter1  =  pdp.pdp_interact(model=tree_model, dataset=X_test, model_features=feature_names, features=features_to_plot)

pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=features_to_plot, plot_type='contour', plot_pdp=True)
plt.show()

# 3. SHAP Values

SHAP which stands for **SH**apley **A**dditive ex**P**lanation, helps to break down a prediction to show the impact of each feature. It is based on Shapley values, a technique used in game theory to determine how much each player in a collaborative game has contributed to its success[¹](https://medium.com/civis-analytics/demystifying-black-box-models-with-shap-value-analysis-3e20b536fc80). Normally, getting the trade-off between accuracy and interpretability just right can be a difficult balancing act but SHAP values can deliver both.

SHAP values interpret the impact of having a certain value for a given feature in comparison to the prediction we’d make if that feature  took  some baseline value.

Shap values show how much a given feature changed our prediction (compared to if we made that prediction at some baseline value of that feature). Let’s say we wanted to know what was the prediction when the insulin level was 150 instead of some fixed baseline no. If we are able to answer this, we could perform the same steps for other features as follows:

`sum(SHAP values for all features) = pred_for_team - pred_for_baseline_values`

In [ ]:
row_to_show = 10
data_for_prediction = X_test.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

tree_model.predict_proba(data_for_prediction_array)

SHAP values are calculated using the [Shap](https://github.com/slundberg/shap) library which can be installed easily from PyPI or conda. SHAP values interpret the impact of having a certain value for a given feature in comparison to the prediction we’d make if that feature took some baseline value.




In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(tree_model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

**Interpretation**

The above explanation shows features each contributing to push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue

-   The base_value here is 0.3576 while our predicted value is 1.
-   `Glucose`  = 158 has the biggest impact on increasing the prediction, while
-   `Age`  feature has the biggest effect in decreasing the prediction.

# 4. Advanced Uses of SHAP Values

Aggregating  many SHAP values can provide even more detailed insights into the model.

-   **SHAP Summary Plots**

To get an overview of which features are most important for a model we can plot the SHAP values of every feature for every sample. The summary plot tells which features are most important, and also their range of effects over the dataset.

In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(tree_model)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_test)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values[1],X_test)

For every dot:

-   Vertical location shows what feature it is depicting
-   Color shows whether that feature was high or low for that row of the dataset
-   Horizontal location shows whether the effect of that value caused a higher or lower prediction.

The point in the upper left was depicts a person whose glucose level is less thereby reducing the prediction of diabetes by 0.4.


# 5. LIME: Locally Interpretable Model-Agnostic Explanations

The framework LIME is introduced in a paper titled: ["Why Should I Trust You?": Explaining the Predictions of Any Classifier](https://arxiv.org/abs/1602.04938). This paper talks about an algorithm that is capable of explaining the predicitons of a ML model(classification/Regression problem) in a reasonable way.LIME essentially approximates the predictions of the blackbox models locally with an intrepretable model.

![](https://miro.medium.com/max/1400/0*42-YCqzaLu4NQpIE.jpg)
LIME attempts to play the role of the ‘explainer’, explaining predictions for each data sample. [Source](https://www.oreilly.com/content/introduction-to-local-interpretable-model-agnostic-explanations-lime/)

To make it more understandable, I shall be using the Titanic dataset for this part of the notebook.

In [ ]:
import lime
from lime.lime_tabular import LimeTabularExplainer

# Reading in the data
df_titanic = pd.read_csv('../input/titanic/train.csv')
df_titanic.drop('PassengerId',axis=1,inplace=True)

# Filling the missing values
df_titanic.fillna(0,inplace=True)

# label encoding categorical data
le = LabelEncoder()

df_titanic['Pclass_le'] = le.fit_transform(df_titanic['Pclass'])
df_titanic['SibSp_le'] = le.fit_transform(df_titanic['SibSp'])
df_titanic['Sex_le'] = le.fit_transform(df_titanic['Sex'])

cat_col = ['Pclass_le', 'Sex_le','SibSp_le', 'Parch','Fare']

# create validation set
X_train,X_test,y_train,y_test = train_test_split(df_titanic[cat_col],df_titanic[['Survived']],test_size=0.3)


# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)


# training the lightgbm model
lgb_params = {
    'task': 'train',
    'boosting_type': 'goss',
    'objective': 'binary',
    'metric':'binary_logloss',
    'metric': {'l2', 'auc'},
    'num_leaves': 50,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'verbose': None,
    'num_iteration':100,
    'num_threads':7,
    'max_depth':12,
    'min_data_in_leaf':100,
    'alpha':0.5}
model = lgb.train(lgb_params,lgb_train,num_boost_round=10,valid_sets=lgb_eval,early_stopping_rounds=5)



# LIME requires class probabilities in case of classification example
def prob(data):
    return np.array(list(zip(1-model.predict(data),model.predict(data))))
    



In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(df_titanic[model.feature_name()].astype(int).values,  
mode='classification',training_labels=df_titanic['Survived'],feature_names=model.feature_name())


# Obtain the explanations from LIME for particular values in the validation dataset
i = 1
exp = explainer.explain_instance(df_titanic.loc[i,cat_col].astype(int).values, prob, num_features=5)

exp.show_in_notebook(show_table=True)

For 1st observation, LIME provides an explanation as to the reason for assigning the particluar probability.The classifier predicts the person's probability for surviving is 64%.The blue color contributes negatively to the positive class and the orange line, positively.

# Conclusion

Machine Learning doesn’t have to be a black box anymore. What use is a good model if we cannot explain the results to others. Interpretability is as important as creating a model.  To achieve wider acceptance among the population, it is crucial that Machine learning systems are able to provide satisfactory explanations for their decisions.  As Albert Einstein said,” **If you can’t explain** it simply, **you** don’t understand it well enough”.


## References :
* [Interpretable Machine Learning: A Guide for Making Black Box Models Explainable.Christoph Molnar](https://christophm.github.io/interpretable-ml-book/)
* [Machine Learning Explainability Micro Course: Kaggle](https://www.kaggle.com/learn/machine-learning-explainability)